<h1 span style="text-align:center;color:mediumvioletred">Tensorflow Input Pipeline</h1>

In [1]:
import tensorflow as tf

In [2]:
daily_sales_numbers = [22,26,32,44,-33,36,28,21,-56,28]

## Make tf_dataset

In [3]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [4]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

22
26
32
44
-33
36
28
21
-56
28


In [5]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

22
26
32


## Filter 

In [6]:
tf_dataset = tf_dataset.filter(lambda x: x>0)

In [7]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

22
26
32
44
36
28
21
28


## Transformation

### 1. Currency Conversion from USD to BDT
    where 1 USD = 121 BDT

In [8]:
tf_dataset = tf_dataset.map(lambda x: x*121)

In [9]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

2662
3146
3872
5324
4356
3388
2541
3388


### 2. Random shuffle

In [10]:
tf_dataset = tf_dataset.shuffle(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

3146
3872
2662
5324
3388
2541
3388
4356


### 3. Batching

In [11]:
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[2662 3146]
[3872 5324]
[4356 2541]
[3388 3388]


# Input Pipeline

In [12]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset = tf_dataset.filter(lambda x: x>0).map(lambda y: y*121).shuffle(2).batch(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[2662 3872]
[5324 3146]
[4356 2541]
[3388 3388]


## Images

In [13]:
images_ds = tf.data.Dataset.list_files('input_pipeline/*/*', shuffle=False)
for file in images_ds.take(5):
    print(file.numpy())

b'input_pipeline\\cat\\gettyimages-1127317526-612x612.jpg'
b'input_pipeline\\cat\\gettyimages-1184147421-612x612.jpg'
b'input_pipeline\\cat\\gettyimages-1194409229-612x612.jpg'
b'input_pipeline\\cat\\gettyimages-1248313892-612x612.jpg'
b'input_pipeline\\cat\\gettyimages-1290233518-612x612.jpg'


In [14]:
images_ds = images_ds.shuffle(200)
for file in images_ds.take(5):
    print(file.numpy())

b'input_pipeline\\dog\\gettyimages-175498455-612x612.jpg'
b'input_pipeline\\cat\\gettyimages-1402118614-612x612.jpg'
b'input_pipeline\\cat\\gettyimages-1290233518-612x612.jpg'
b'input_pipeline\\cat\\gettyimages-1387641909-612x612.jpg'
b'input_pipeline\\cat\\gettyimages-1194409229-612x612.jpg'


In [15]:
class_names = ["cat", "dog"]

In [16]:
image_count = len(images_ds)
image_count

100

In [17]:
train_size = int(image_count*0.8)

train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [18]:
len(train_ds)

80

In [19]:
len(test_ds)

20

In [20]:
path = 'input_pipeline\\cat\\gettyimages-1127317526-612x612.jpg'
path.split("\\")[-2]

'cat'

In [21]:
def get_label(path):
    import os
    return tf.strings.split(path, os.path.sep)[-2]

In [22]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128,128])
    return img, label

In [23]:
for label in train_ds.map(get_label):
    print(label.numpy())

b'cat'
b'dog'
b'dog'
b'cat'
b'dog'
b'dog'
b'cat'
b'dog'
b'dog'
b'dog'
b'cat'
b'cat'
b'cat'
b'dog'
b'dog'
b'cat'
b'dog'
b'cat'
b'dog'
b'dog'
b'dog'
b'dog'
b'cat'
b'dog'
b'dog'
b'cat'
b'dog'
b'dog'
b'cat'
b'cat'
b'dog'
b'dog'
b'cat'
b'dog'
b'dog'
b'cat'
b'dog'
b'dog'
b'cat'
b'cat'
b'dog'
b'cat'
b'cat'
b'dog'
b'dog'
b'dog'
b'cat'
b'cat'
b'cat'
b'cat'
b'cat'
b'cat'
b'cat'
b'dog'
b'dog'
b'cat'
b'dog'
b'cat'
b'dog'
b'dog'
b'dog'
b'cat'
b'dog'
b'cat'
b'cat'
b'cat'
b'cat'
b'dog'
b'cat'
b'dog'
b'cat'
b'dog'
b'cat'
b'cat'
b'cat'
b'dog'
b'cat'
b'cat'
b'dog'
b'dog'


In [24]:
for t in train_ds.take(5):
    print(t.numpy())

b'input_pipeline\\cat\\gettyimages-1392773192-612x612.jpg'
b'input_pipeline\\dog\\gettyimages-171155760-612x612.jpg'
b'input_pipeline\\cat\\gettyimages-2151625435-612x612.jpg'
b'input_pipeline\\cat\\gettyimages-937809006-612x612.jpg'
b'input_pipeline\\cat\\gettyimages-1498838344-612x612.jpg'


In [25]:
train_ds = train_ds.map(process_image)
for img, label in train_ds.take(3):
     print(img)
     print(label.numpy())

tf.Tensor(
[[[137.30078   173.7539     13.020874 ]
  [ 59.751648   88.969055    5.8658447]
  [ 51.036804   78.97839     3.6340942]
  ...
  [143.75763   157.77307    60.94641  ]
  [116.79175   127.74945    60.247437 ]
  [105.97113   115.522766   48.582703 ]]

 [[ 96.15491   131.17767     9.800903 ]
  [ 90.565735  128.1518     25.753113 ]
  [ 59.05243    89.51782     1.6193848]
  ...
  [165.54749   184.80139    67.15405  ]
  [134.1579    146.44305    70.48804  ]
  [102.429016  109.47131    57.12567  ]]

 [[ 60.13153    80.470825   11.8932495]
  [101.47766   123.45477    54.89624  ]
  [ 79.5246    106.83252    10.570251 ]
  ...
  [129.94507   138.66174    71.18451  ]
  [131.93976   136.36163    92.96765  ]
  [121.54425   128.6925     75.64172  ]]

 ...

 [[110.54584   134.97998    73.40912  ]
  [ 87.411865   73.19702    51.10553  ]
  [134.96533   175.68225    54.132385 ]
  ...
  [ 89.88934    82.5672     66.72827  ]
  [104.53784    97.53784    81.53784  ]
  [ 95.12775    83.12775    69.12

In [26]:
def scale(image, label):
    return image/255, label

In [27]:
train_ds = train_ds.map(scale)

In [28]:
for image, label in train_ds.take(5):
    print("****Image: ", image.numpy()[0][0])
    print("****Label", label.numpy())

****Image:  [0.12898284 0.14859068 0.16035539]
****Label b'cat'
****Image:  [0.9159926 0.956924  0.9939338]
****Label b'cat'
****Image:  [0.98078084 0.9847024  0.96509457]
****Label b'dog'
****Image:  [0.3878064  0.34859067 0.309375  ]
****Label b'cat'
****Image:  [0.78431374 0.7490196  0.72156864]
****Label b'cat'
